In [1]:
import sys
sys.path.insert(0, r'H:\.Workspace\22_SUMMER\PINNs-master\Utilities')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
from plotting import newfig, savefig
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math

d:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:522: Futu

In [2]:
a=np.array([
    [1,2,3],
    [2,3,4],
    [7,8,9]
])
np.reshape(a[0:2],(6,1))

array([[1],
       [2],
       [3],
       [2],
       [3],
       [4]])

In [5]:
 tmp = np.float32(np.loadtxt(r'H:\.Workspace\22_SUMMER\PINNs-master\Utilities\IRK_weights\Butcher_IRK%d.txt' % (5), ndmin = 2))
 tmp.shape

(35, 1)

In [8]:
data = scipy.io.loadmat(r'H:\.Workspace\22_SUMMER\PINNs-master\main\Data\AC.mat')
t = data['tt'].flatten()[:,None] # T x 1
x = data['x'].flatten()[:,None] # N x 1
idx_t0 = 20
idx_t1 = 180
dt = t[idx_t1] - t[idx_t0]
# print(dt)
# print(t[idx_t1])
# print(t[idx_t0])
print(t)

[[0.   ]
 [0.005]
 [0.01 ]
 [0.015]
 [0.02 ]
 [0.025]
 [0.03 ]
 [0.035]
 [0.04 ]
 [0.045]
 [0.05 ]
 [0.055]
 [0.06 ]
 [0.065]
 [0.07 ]
 [0.075]
 [0.08 ]
 [0.085]
 [0.09 ]
 [0.095]
 [0.1  ]
 [0.105]
 [0.11 ]
 [0.115]
 [0.12 ]
 [0.125]
 [0.13 ]
 [0.135]
 [0.14 ]
 [0.145]
 [0.15 ]
 [0.155]
 [0.16 ]
 [0.165]
 [0.17 ]
 [0.175]
 [0.18 ]
 [0.185]
 [0.19 ]
 [0.195]
 [0.2  ]
 [0.205]
 [0.21 ]
 [0.215]
 [0.22 ]
 [0.225]
 [0.23 ]
 [0.235]
 [0.24 ]
 [0.245]
 [0.25 ]
 [0.255]
 [0.26 ]
 [0.265]
 [0.27 ]
 [0.275]
 [0.28 ]
 [0.285]
 [0.29 ]
 [0.295]
 [0.3  ]
 [0.305]
 [0.31 ]
 [0.315]
 [0.32 ]
 [0.325]
 [0.33 ]
 [0.335]
 [0.34 ]
 [0.345]
 [0.35 ]
 [0.355]
 [0.36 ]
 [0.365]
 [0.37 ]
 [0.375]
 [0.38 ]
 [0.385]
 [0.39 ]
 [0.395]
 [0.4  ]
 [0.405]
 [0.41 ]
 [0.415]
 [0.42 ]
 [0.425]
 [0.43 ]
 [0.435]
 [0.44 ]
 [0.445]
 [0.45 ]
 [0.455]
 [0.46 ]
 [0.465]
 [0.47 ]
 [0.475]
 [0.48 ]
 [0.485]
 [0.49 ]
 [0.495]
 [0.5  ]
 [0.505]
 [0.51 ]
 [0.515]
 [0.52 ]
 [0.525]
 [0.53 ]
 [0.535]
 [0.54 ]
 [0.545]
 [0.55 ]
 

In [12]:
Exact = np.real(data['uu']).T
N=200
idx_x = np.random.choice(Exact.shape[1], N, replace=False)
u0 = Exact[idx_t0:idx_t0+1,idx_x].T
print(u0)

[[-3.55393195e-01]
 [-1.05384049e-02]
 [ 7.87537411e-02]
 [-7.34560140e-01]
 [ 7.13316496e-02]
 [-2.82018561e-01]
 [-7.08113365e-01]
 [-9.90930249e-01]
 [-8.06694715e-01]
 [ 6.34566795e-03]
 [-5.24158778e-03]
 [ 5.61782353e-02]
 [ 7.27214431e-02]
 [-8.73854197e-01]
 [-6.22294161e-01]
 [ 9.06315292e-02]
 [-5.91818697e-01]
 [-1.17090858e-01]
 [-2.03632224e-01]
 [-3.23527448e-01]
 [ 7.51754767e-02]
 [ 7.13316496e-02]
 [-8.48615992e-01]
 [ 1.85900441e-02]
 [ 4.65528853e-02]
 [-2.82018561e-01]
 [ 5.61782353e-02]
 [-8.55126043e-01]
 [ 2.24633591e-02]
 [ 2.32353253e-02]
 [-1.67327527e-01]
 [ 8.44882519e-02]
 [ 5.57495571e-02]
 [ 9.78820076e-03]
 [-9.70395551e-01]
 [ 4.33502209e-02]
 [-5.71079504e-01]
 [ 8.78755396e-02]
 [-5.39437710e-01]
 [-1.33246206e-01]
 [ 7.54046584e-02]
 [ 1.63433836e-03]
 [ 1.50119985e-02]
 [ 5.93410323e-02]
 [ 6.99126352e-02]
 [ 5.27148595e-02]
 [ 6.69955697e-02]
 [-2.51814269e-01]
 [-5.21075836e-02]
 [-6.54344695e-02]
 [ 2.80270902e-02]
 [ 9.03123451e-02]
 [-1.5859168

In [15]:
a=np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
])
b=a[0:-1,:]
print(b)
c=a[-1:,:]
print(c)

[[1 2 3]
 [4 5 6]]
[[7 8 9]]


In [23]:
q=5
tmp = np.float32(np.loadtxt(r'H:\.Workspace\22_SUMMER\PINNs-master\Utilities\IRK_weights\Butcher_IRK%d.txt' % (q), ndmin = 2))
IRK_weights = np.reshape(tmp[0:q**2+q], (q+1,q))
IRK_times = tmp[q**2+q:] #shape = (q,1)
print(IRK_times)

[[0.04691008]
 [0.23076534]
 [0.5       ]
 [0.76923466]
 [0.95308995]]


In [24]:
1/5

0.2